<a href="https://colab.research.google.com/github/mvluzzi/BEONtech/blob/main/BEON_Airport_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Input data**

In [1]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F

spark = SparkSession.builder.master("local[*]").appName("ColabSpark").getOrCreate()

In [6]:
nyc_airlines_path = "/content/drive/MyDrive/BEON.tech/nyc_airlines.csv"
nyc_airports_path = "/content/drive/MyDrive/BEON.tech/nyc_airports.csv"
nyc_flights_path = "/content/drive/MyDrive/BEON.tech/nyc_flights_fixed.csv"
nyc_planes_path = "/content/drive/MyDrive/BEON.tech/nyc_planes.csv"
nyc_weather_path = "/content/drive/MyDrive/BEON.tech/nyc_weather.csv"

airlines = spark.read.csv(nyc_airlines_path, header=True)
airports = spark.read.csv(nyc_airports_path, header=True)
flights = spark.read.csv(nyc_flights_path, header=True)
planes = spark.read.csv(nyc_planes_path, header=True)
weather = spark.read.csv(nyc_weather_path, header=True)
#mockdata.write.mode("overwrite").saveAsTable("mockdata")

# Overalpane
1. Determine the number of distinct destinations connected to the airport.
2. Calculate the rounded average of distinct destinations for each day.
3. Calculate the average number of destinations per day of the week.
4. Identify the month with the highest number of flights.

## 1.

In [12]:
flights.groupBy("origin").agg(F.countDistinct("dest")).show()

+------+--------------------+
|origin|count(DISTINCT dest)|
+------+--------------------+
|   LGA|                  68|
|   EWR|                  82|
|   JFK|                  65|
+------+--------------------+



# 2.

In [25]:
flights.select("dest", F.to_date("time_hour").alias("date")).groupBy("date").agg(F.countDistinct("dest").alias("distinct_dest")).agg(F.round(F.avg("distinct_dest")).alias("rounded_average_flights")).show()

+-----------------------+
|rounded_average_flights|
+-----------------------+
|                   81.0|
+-----------------------+



# 3

In [33]:
flights.select(F.weekday(F.to_date("time_hour")).alias("week_day"), "dest").groupBy("week_day").agg(F.countDistinct("dest")).show()

+--------+--------------------+
|week_day|count(DISTINCT dest)|
+--------+--------------------+
|       1|                  92|
|       6|                  95|
|       3|                  93|
|       5|                  98|
|       4|                  94|
|       2|                  92|
|       0|                  93|
+--------+--------------------+



# Delays
5. Identify the month with the highest accumulated departure delay.
6. Analyze the correlation between departure delay and weather conditions by cross-referencing
the departure delay data with the weather dataset.

In [41]:
from pyspark.sql.window import Window

In [48]:
flights.where(F.col("dep_delay").cast("int")>0).select("month", "_c0").groupBy("month").agg(F.count("_c0").alias("delayed_flights")).withColumn("rank", F.row_number().over(Window.partitionBy(F.lit(1)).orderBy(F.desc("delayed_flights")))).where(F.col("rank")==1).show()

+-----+---------------+----+
|month|delayed_flights|rank|
+-----+---------------+----+
|    7|          11841|   1|
+-----+---------------+----+



# Airlines
7. Create a ranking of airlines based on the number of flights they operate.
8. Create a ranking of airlines based on the daily availability of seats.

# Airplanes
9. Determine the most common plane manufacturer.
10. Identify the most common plane model.

# **Write on Drive and create CSV**

In [ ]:
mock.coalesce(1).write.mode("overwrite").format("csv").save("/content/drive/MyDrive/transformed_spark_output")